In [1]:
import pandas as pd
import numpy as np
import math
from time import time
from pyspark import SparkContext 
from pyspark import SparkConf 
import sys
import matplotlib.pyplot as plt

In [2]:
input_train_business = "/Users/ar1607/Desktop/USC/FourthSem/finalProj/Restaurants_NEW/anshul_business_train_PA.csv"
input_test_business = "/Users/ar1607/Desktop/USC/FourthSem/finalProj/Restaurants_NEW/anshul_business_test_PA.csv"
input_validation_business = "/Users/ar1607/Desktop/USC/FourthSem/finalProj/Restaurants_NEW/anshul_business_val_PA.csv"
inputFile = "/Users/ar1607/Desktop/USC/FourthSem/finalProj/Restaurants_NEW/train/PA_train_yelp_academic_dataset_review.csv"
testFile = "/Users/ar1607/Desktop/USC/FourthSem/finalProj/Restaurants_NEW/test/PA_test_yelp_academic_dataset_review.csv"
valFile = "/Users/ar1607/Desktop/USC/FourthSem/finalProj/Restaurants_NEW/valid/PA_valid_yelp_academic_dataset_review.csv"

In [3]:
#sc = SparkContext('local[2]', 'ItemBasedCF')
conf = SparkConf().setAppName("ItemBasedCF")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '4G')
        .set('spark.driver.memory', '45G')
        .set('spark.driver.maxResultSize', '10G'))
sc = SparkContext(conf=conf)

In [4]:
def _hash(a):
    return hash(a)%100000000

In [5]:
#created Map of businessID hash to one-hot-vector of attributes
test_data_one_hot = sc.textFile(input_test_business)
test_header_one_hot = test_data_one_hot.first()
test_data_one_hot = test_data_one_hot.filter(lambda line: line != test_header_one_hot).map(lambda line: line.split(','))\
            .map(lambda line: (_hash(line[0]), 0 if line[1]=='' else line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], 1 if line[1]=='True' else line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], 0 if line[1]=='False' else line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], 0 if line[2]=='' else line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], 1 if line[2]=='True' else line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], 0 if line[2]=='False' else line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], 0 if line[3]=='' else line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], 1 if line[3]=='True' else line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], 0 if line[3]=='False' else line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], 0 if line[4]=='' else line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], 1 if line[4]=='True' else line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], 0 if line[4]=='False' else line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], 0 if line[5]=='' else line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], 1 if line[5]=='True' else line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], 0 if line[5]=='False' else line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], 0 if line[6]=='' else line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], 1 if line[6]=='True' else line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], 0 if line[6]=='False' else line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], 0 if line[7]=='' else line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], 1 if line[7]=='True' else line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], 0 if line[7]=='False' else line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], 0 if line[8]=='' else line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], 1 if line[8]=='True' else line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], 0 if line[8]=='False' else line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], 0 if line[9]=='' else line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], 1 if line[9]=='True' else line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], 0 if line[9]=='False' else line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], 0 if line[10]=='' else line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], 1 if line[10]=='True' else line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], 0 if line[10]=='False' else line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], 0 if line[11]=='' else line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], 1 if line[11]=='True' else line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], 0 if line[11]=='False' else line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], 0 if line[12]=='' else line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], 1 if line[12]=='True' else line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], 0 if line[12]=='False' else line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], 0.0 if line[13]=='' else line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], 1.0 if line[13]=='none' else line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], 2.0 if line[13]=='beer_and_wine' else line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], 3.0 if line[13]=='full_bar' else line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 0.0 if line[14]=='' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 1.0 if line[14]=='quiet' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 2.0 if line[14]=='average' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 3.0 if line[14]=='very_loud' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 4.0 if line[14]=='loud' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], 0.0 if line[15]=='' else float(line[15]), line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], 0.0 if line[16]=='' else float(line[16])))
            
print "TEST DATA : \n", test_data_one_hot.take(10)

train_data_one_hot = sc.textFile(input_train_business)
train_header_one_hot = train_data_one_hot.first()
xx = train_data_one_hot.map(lambda line: line.split(','))#.map(lambda line : (line[82], line[4], line[20], line[22], line[25], line[60], line[73], line[45], line[46], line[47], line[48], line[49], line[50], line[3], line[68], line[75]))

train_data_one_hot = train_data_one_hot.filter(lambda line: line != train_header_one_hot).map(lambda line: line.split(','))\
            .map(lambda line: (_hash(line[0]), 0 if line[1]=='' else line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], 1 if line[1]=='True' else line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], 0 if line[1]=='False' else line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], 0 if line[2]=='' else line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], 1 if line[2]=='True' else line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], 0 if line[2]=='False' else line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], 0 if line[3]=='' else line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], 1 if line[3]=='True' else line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], 0 if line[3]=='False' else line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], 0 if line[4]=='' else line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], 1 if line[4]=='True' else line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], 0 if line[4]=='False' else line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], 0 if line[5]=='' else line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], 1 if line[5]=='True' else line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], 0 if line[5]=='False' else line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], 0 if line[6]=='' else line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], 1 if line[6]=='True' else line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], 0 if line[6]=='False' else line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], 0 if line[7]=='' else line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], 1 if line[7]=='True' else line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], 0 if line[7]=='False' else line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], 0 if line[8]=='' else line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], 1 if line[8]=='True' else line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], 0 if line[8]=='False' else line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], 0 if line[9]=='' else line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], 1 if line[9]=='True' else line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], 0 if line[9]=='False' else line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], 0 if line[10]=='' else line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], 1 if line[10]=='True' else line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], 0 if line[10]=='False' else line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], 0 if line[11]=='' else line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], 1 if line[11]=='True' else line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], 0 if line[11]=='False' else line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], 0 if line[12]=='' else line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], 1 if line[12]=='True' else line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], 0 if line[12]=='False' else line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], 0.0 if line[13]=='' else line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], 1.0 if line[13]=='none' else line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], 2.0 if line[13]=='beer_and_wine' else line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], 3.0 if line[13]=='full_bar' else line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 0.0 if line[14]=='' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 1.0 if line[14]=='quiet' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 2.0 if line[14]=='average' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 3.0 if line[14]=='very_loud' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 4.0 if line[14]=='loud' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], 0.0 if line[15]=='' else float(line[15]), line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], 0.0 if line[16]=='' else float(line[16])))
            
            
print "\nTRAIN DATA : \n", train_data_one_hot.take(10)
xx = xx.filter(lambda line : line[15]=='True')  
print "\n ",xx.take(1)

val_data_one_hot = sc.textFile(input_validation_business)
val_header_one_hot = val_data_one_hot.first()
val_data_one_hot = val_data_one_hot.filter(lambda line: line != val_header_one_hot).map(lambda line: line.split(','))\
            .map(lambda line: (_hash(line[0]), 0 if line[1]=='' else line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], 1 if line[1]=='True' else line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], 0 if line[1]=='False' else line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], 0 if line[2]=='' else line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], 1 if line[2]=='True' else line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], 0 if line[2]=='False' else line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], 0 if line[3]=='' else line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], 1 if line[3]=='True' else line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], 0 if line[3]=='False' else line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], 0 if line[4]=='' else line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], 1 if line[4]=='True' else line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], 0 if line[4]=='False' else line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], 0 if line[5]=='' else line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], 1 if line[5]=='True' else line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], 0 if line[5]=='False' else line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], 0 if line[6]=='' else line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], 1 if line[6]=='True' else line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], 0 if line[6]=='False' else line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], 0 if line[7]=='' else line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], 1 if line[7]=='True' else line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], 0 if line[7]=='False' else line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], 0 if line[8]=='' else line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], 1 if line[8]=='True' else line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], 0 if line[8]=='False' else line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], 0 if line[9]=='' else line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], 1 if line[9]=='True' else line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], 0 if line[9]=='False' else line[9], line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], 0 if line[10]=='' else line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], 1 if line[10]=='True' else line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], 0 if line[10]=='False' else line[10], line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], 0 if line[11]=='' else line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], 1 if line[11]=='True' else line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], 0 if line[11]=='False' else line[11], line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], 0 if line[12]=='' else line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], 1 if line[12]=='True' else line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], 0 if line[12]=='False' else line[12], line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], 0.0 if line[13]=='' else line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], 1.0 if line[13]=='none' else line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], 2.0 if line[13]=='beer_and_wine' else line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], 3.0 if line[13]=='full_bar' else line[13], line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 0.0 if line[14]=='' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 1.0 if line[14]=='quiet' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 2.0 if line[14]=='average' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 3.0 if line[14]=='very_loud' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], 4.0 if line[14]=='loud' else line[14], line[15], line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], 0.0 if line[15]=='' else float(line[15]), line[16]))\
            .map(lambda line: (line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13], line[14], line[15], 0.0 if line[16]=='' else float(line[16])))
            
print "\nVAL DATA : \n", val_data_one_hot.take(10)
print "\ntrain vector size : \n", train_data_one_hot.count()

TEST DATA : 
[(45206826, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 1.0, 2.0), (43466363, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 3.0, 2.0, 2.0, 3.0), (2201188, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 2.0, 1.5), (83227545, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1.0, 2.0, 2.0, 4.5), (74268317, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1.0, 2.0, 1.0, 2.5), (85385037, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 0.0, 5.0), (80452614, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.0, 2.0, 2.0, 3.5), (96568505, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0.0, 2.0, 1.0, 4.0), (87057478, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1.0, 0.0, 1.0, 4.0), (76660619, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3.0, 2.0, 2.0, 2.0)]

TRAIN DATA : 
[(45206826, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 1.0, 2.0), (43466363, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 3.0, 2.0, 2.0, 3.0), (2201188, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 2.0, 1.5), (83227545, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1.0, 2.0, 2.0, 4.5), (97673544, 0, 1, 1, 

In [6]:
"""
trn = pd.read_csv(inputFile)
del trn['text']
trn.to_csv("./anshul_review_train_PA.csv", encoding='utf-8', index=False)
val = pd.read_csv(valFile)
del val['text']
val.to_csv("./anshul_review_val_PA.csv", encoding='utf-8', index=False)
tst = pd.read_csv(testFile)
del tst['text']
tst.to_csv("./anshul_review_test_PA.csv", encoding='utf-8', index=False)
"""

inputFile = "./anshul_review_train_PA.csv"
valFile = "./anshul_review_val_PA.csv"
testFile = "./anshul_review_test_PA.csv"

"""
trn = pd.read_csv("/Users/ar1607/Desktop/USC/FourthSem/finalProj/Restaurants_NEW/train/PA_train_yelp_academic_dataset_business.csv")
new_trn = trn.filter(['business_id','attributes.Ambience.casual','attributes.BikeParking','attributes.BusinessAcceptsCreditCards','attributes.BusinessParking.street','attributes.HasTV','attributes.RestaurantsDelivery','attributes.GoodForMeal.breakfast','attributes.GoodForMeal.brunch','attributes.GoodForMeal.dessert','attributes.GoodForMeal.dinner','attributes.GoodForMeal.latenight','attributes.GoodForMeal.lunch','attributes.Alcohol','attributes.NoiseLevel','attributes.RestaurantsPriceRange2','stars'], axis=1)
new_trn.to_csv("./anshul_business_train_PA.csv", encoding='utf-8', index=False)

val = pd.read_csv("/Users/ar1607/Desktop/USC/FourthSem/finalProj/Restaurants_NEW/valid/PA_valid_yelp_academic_dataset_business.csv")
new_val = val.filter(['business_id','attributes.Ambience.casual','attributes.BikeParking','attributes.BusinessAcceptsCreditCards','attributes.BusinessParking.street','attributes.HasTV','attributes.RestaurantsDelivery','attributes.GoodForMeal.breakfast','attributes.GoodForMeal.brunch','attributes.GoodForMeal.dessert','attributes.GoodForMeal.dinner','attributes.GoodForMeal.latenight','attributes.GoodForMeal.lunch','attributes.Alcohol','attributes.NoiseLevel','attributes.RestaurantsPriceRange2','stars'], axis=1)
new_val.to_csv("./anshul_business_val_PA.csv", encoding='utf-8', index=False)

tst = pd.read_csv("/Users/ar1607/Desktop/USC/FourthSem/finalProj/Restaurants_NEW/test/PA_test_yelp_academic_dataset_business.csv")
new_tst = tst.filter(['business_id','attributes.Ambience.casual','attributes.BikeParking','attributes.BusinessAcceptsCreditCards','attributes.BusinessParking.street','attributes.HasTV','attributes.RestaurantsDelivery','attributes.GoodForMeal.breakfast','attributes.GoodForMeal.brunch','attributes.GoodForMeal.dessert','attributes.GoodForMeal.dinner','attributes.GoodForMeal.latenight','attributes.GoodForMeal.lunch','attributes.Alcohol','attributes.NoiseLevel','attributes.RestaurantsPriceRange2','stars'], axis=1)
new_tst.to_csv("./anshul_business_test_PA.csv", encoding='utf-8', index=False)
"""

'\ntrn = pd.read_csv("/Users/ar1607/Desktop/USC/FourthSem/finalProj/Restaurants_NEW/train/PA_train_yelp_academic_dataset_business.csv")\nnew_trn = trn.filter([\'business_id\',\'attributes.Ambience.casual\',\'attributes.BikeParking\',\'attributes.BusinessAcceptsCreditCards\',\'attributes.BusinessParking.street\',\'attributes.HasTV\',\'attributes.RestaurantsDelivery\',\'attributes.GoodForMeal.breakfast\',\'attributes.GoodForMeal.brunch\',\'attributes.GoodForMeal.dessert\',\'attributes.GoodForMeal.dinner\',\'attributes.GoodForMeal.latenight\',\'attributes.GoodForMeal.lunch\',\'attributes.Alcohol\',\'attributes.NoiseLevel\',\'attributes.RestaurantsPriceRange2\',\'stars\'], axis=1)\nnew_trn.to_csv("./anshul_business_train_PA.csv", encoding=\'utf-8\', index=False)\n\nval = pd.read_csv("/Users/ar1607/Desktop/USC/FourthSem/finalProj/Restaurants_NEW/valid/PA_valid_yelp_academic_dataset_business.csv")\nnew_val = val.filter([\'business_id\',\'attributes.Ambience.casual\',\'attributes.BikeParking\

In [7]:
ratings_train_data = sc.textFile(inputFile)
ratings_train_data_header = ratings_train_data.first()
ratings_train_data = ratings_train_data.filter(lambda line: line != ratings_train_data_header)\
                    .map(lambda line: line.split(','))\
                    .map(lambda line: ((_hash(line[3]), _hash(line[1])), float(line[4])))
                    #.map(lambda line: ((line[3], line[1]), float(line[4])))
print "\nTRAIN DATA : \n", ratings_train_data.take(10)

ratings_val_data = sc.textFile(valFile)
ratings_val_data_header = ratings_val_data.first()
ratings_val_data = ratings_val_data.filter(lambda line: line != ratings_val_data_header)\
                    .map(lambda line: line.split(','))\
                    .map(lambda line: ((_hash(line[3]), _hash(line[1])), float(line[4])))
                    #.map(lambda line: ((line[3], line[1]), float(line[4])))
print "\nVAL DATA : \n", ratings_val_data.take(10)

ratings_test_data = sc.textFile(testFile)
ratings_test_data_header = ratings_test_data.first()
ratings_test_data = ratings_test_data.filter(lambda line: line != ratings_test_data_header)\
                    .map(lambda line: line.split(','))\
                    .map(lambda line: ((_hash(line[3]), _hash(line[1])), float(line[4])))
                    #.map(lambda line: ((line[3], line[1]), float(line[4])))
print "\nTEST DATA : \n", ratings_test_data.take(10)
print "count test : ", ratings_test_data.count()


TRAIN DATA : 
[((95456344, 16468096), 4.0), ((15550168, 5713139), 4.0), ((51998740, 30916773), 3.0), ((10099958, 30916773), 4.0), ((87905139, 30916773), 4.0), ((31626270, 30916773), 4.0), ((39111954, 30916773), 4.0), ((314381, 30916773), 1.0), ((26797418, 30916773), 4.0), ((46147123, 30916773), 5.0)]

VAL DATA : 
[((47719706, 16468096), 5.0), ((4084006, 16468096), 5.0), ((79786736, 30916773), 4.0), ((882477, 30916773), 5.0), ((11411919, 30916773), 3.0), ((59804841, 95762994), 5.0), ((65161490, 52554862), 1.0), ((9484790, 52554862), 5.0), ((31759712, 15591569), 5.0), ((13392708, 99671206), 4.0)]

TEST DATA : 
[((56668287, 30916773), 4.0), ((91365596, 30916773), 2.0), ((44755039, 93819697), 5.0), ((24397155, 93819697), 3.0), ((77783735, 11191924), 2.0), ((31042311, 11191924), 4.0), ((95873029, 39020287), 4.0), ((52943122, 39020287), 4.0), ((84320461, 46234978), 4.0), ((86494668, 46234978), 4.0)]
count test :  35611


In [8]:
usrBizDict = ratings_train_data.map(lambda x: tuple((x[0][0], {x[0][1]: x[1]})))\
    .reduceByKey(lambda a, b: dict(a.items() + b.items()))
#print usrBizDict.take(10)

In [9]:
#lines.saveAsTextFile()
#np_train_data_one_hot = genfromtxt('train_ohv.csv', delimiter=',')
#print np_train_data_one_hot
import math

def calculateSimilarityScore(a, b):
    res = 0
    n = len(a)
    numerator, den_A, den_B = 0,0,0
    for i in range(n):
        numerator += (a[i]*b[i])
        den_A += a[i]**2
        den_B += b[i]**2
    denominator = math.sqrt(den_A) * math.sqrt(den_B)
    if denominator==0.0:
        return 0.0
    return numerator/denominator


#union_of_two = train_data_one_hot.union(val_data_one_hot)
#union_of_all = union_of_two.union(test_data_one_hot)
train_data_simi_matrix_tmp = train_data_one_hot.cartesian(train_data_one_hot)#union_of_all.cartesian(union_of_all)#
train_data_simi_matrix = train_data_simi_matrix_tmp.map(lambda line : ((line[0][0], line[1][0]), calculateSimilarityScore(line[0][1:], line[1][1:])))

train_data_simi_matrix = train_data_simi_matrix.filter(lambda x : x[0][0]!=x[0][1])\
                        .map(lambda x: tuple((x[0][0], {x[0][1]: x[1]})))\
                        .reduceByKey(lambda a, b: dict(a.items() + b.items()))
#print train_data_simi_matrix.take(10)
#print train_data_simi_matrix.count()

In [10]:
#biz_to_biz_simi = train_data_simi_matrix.map(lambda x: tuple((x[0][0], {x[0][1]: x[1]})))\
#    .reduceByKey(lambda a, b: dict(a.items() + b.items()))
utilityDict = sc.broadcast(usrBizDict.collectAsMap())
similarityMatrix = sc.broadcast(train_data_simi_matrix.collectAsMap()) #-- IMPLMENT THE NUMPY SOLUTION
#biz2BizSimiMatrix = sc.broadcast(biz_to_biz_simi.collectAsMap())

In [11]:
test_data_ohv = test_data_one_hot.map(lambda line: (line[0],(line)))
print test_data_ohv.take(5)
train_data_ohv = train_data_one_hot.map(lambda line: (line[0],(line)))
print test_data_ohv.take(5)
val_data_ohv = val_data_one_hot.map(lambda line: (line[0],(line)))
print val_data_ohv.take(5)
#union_tmp = train_data_ohv.union(test_data_ohv)
#all_businesses = union_tmp.union(val_data_ohv)
#biz_data_ohv = sc.broadcast(all_businesses.collectAsMap())

[(45206826, (45206826, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 1.0, 2.0)), (43466363, (43466363, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 3.0, 2.0, 2.0, 3.0)), (2201188, (2201188, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 2.0, 1.5)), (83227545, (83227545, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1.0, 2.0, 2.0, 4.5)), (74268317, (74268317, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1.0, 2.0, 1.0, 2.5))]
[(45206826, (45206826, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 1.0, 2.0)), (43466363, (43466363, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 3.0, 2.0, 2.0, 3.0)), (2201188, (2201188, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 2.0, 1.5)), (83227545, (83227545, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1.0, 2.0, 2.0, 4.5)), (74268317, (74268317, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1.0, 2.0, 1.0, 2.5))]
[(45206826, (45206826, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0.0, 1.0, 2.0)), (43466363, (43466363, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 3.0, 2.0, 2.0, 3.0)), (2201188, (2201188, 0, 0, 1, 0, 0, 0, 0, 

In [12]:
#now take the test data and for the test data, create predictions. test data is of the format ((business, user), rating).
#ratings_test_data
#import operator
def predictedRating(record,utilityDictObj,similarityMatrixObj):
    #record is a tuple of (business,user)
    #user to Business Rating -> usrBizDict [FORMAT : (user1, {business1:rating1, business2:rating2,.....})]
    #similarity matrix -> train_data_simi_matrix
    business,user = record[0],record[1]
    utilityDict, similarityMatrix= utilityDictObj.value, similarityMatrixObj.value
    #find the similar businesses to our current business from the usrBizDict
    similar_biz = utilityDict.get(user, None)#.filter(lambda line: line[0][1]==user)
    if similar_biz==None:
        print "ERROR-- user doesn't exist"
        #then we need to find the average of the similarities of the business's similar businesses
        if business not in similarityMatrix:
            return 3.0
        sim_biz = similarityMatrix[business]
        count=0
        simi_sum = 0
        for i in sorted(sim_biz.items(), key=lambda x: x[1], reverse=True):
            if i[0]!=business:
                simi_sum += i[1]
                count+=1
                if count==10: #5->0.330467540921
                    break
        return (simi_sum*3.0)/10 #5->0.330467540921
            
        """n = 0
        simi_sum = 0
        res = []
        for i in similarityMatrix:
            if i[0]==business or i[1]==business:
                n+=1
                if len(res)==0:
                    res.append(similarityMatrix[i])
                else:
                    res.sort()
                    if similarityMatrix[i]>res[0]:
                        if len(res)==10:
                            res.pop(0)
                        res.append(similarityMatrix[i])
                #simi_sum = similarityMatrix[i]
        if n==0:
            return 0.0
        #ss = ','.join([str(i) for i in res])
        return (sum(res)*3.5)/10#simi_sum/n"""
    #similar_biz = similar_biz.take(1).collect()
    #similar_biz = similar_biz[0][1]
    else:
        #user exists
        if business not in similarityMatrix:
            n = len(similar_biz)
            return sum(similar_biz.values())/n
        
        sim_biz = similarityMatrix[business]
        user_rated_businesses = set(similar_biz.keys())
        all_similar_businesses = set(sim_biz.keys())
        common_biz = user_rated_businesses.intersection(all_similar_businesses)
        num, den = 0,0
        count = 0
        for i in common_biz:
            den += sim_biz[i]
            num += (sim_biz[i] * similar_biz[i])
            count+=1
            if count==10:
                break
        if den==0:
            return sum(similar_biz.values())/len(similar_biz.keys())
        return num/10

In [13]:
ratings_test_result = ratings_test_data.map(lambda line: ((line[0][0], line[0][1]), line[1]/5, predictedRating(line[0],utilityDict,similarityMatrix)/5))
print ratings_test_result.take(10)

[((56668287, 30916773), 0.8, 0.5873977232662447), ((91365596, 30916773), 0.4, 0.5966273941367516), ((44755039, 93819697), 1.0, 0.5901637111563038), ((24397155, 93819697), 0.6, 0.5977072490170497), ((77783735, 11191924), 0.4, 0.5962779761931708), ((31042311, 11191924), 0.8, 0.5956518001966173), ((95873029, 39020287), 0.8, 0.5879289668514917), ((52943122, 39020287), 0.8, 0.584570160695076), ((84320461, 46234978), 0.8, 0.5996775546466881), ((86494668, 46234978), 0.8, 0.5907631577050649)]


In [14]:
RMSE = (ratings_test_result.map(lambda x: (x[1] - x[2])**2).mean())**0.5
print "RMSE: " + str(RMSE)

RMSE: 0.322686760085


In [15]:
def findAttrSimilarity(business, curr_attr, similarityMatrixObj,AttrMatrixObj):
    similarityMatrix, AttrMatrix = similarityMatrixObj.value, AttrMatrixObj.value
    s = similarityMatrix[business]
    f = AttrMatrix[business]
    res = 0
    for i in s:
        if f[i][curr_attr]==0:
            res-=s[i]
        else:
            res+=s[i]
    if res<0:
        return 0
    return 1

In [16]:
train_first_attr_matrix = train_data_simi_matrix_tmp.map(lambda x : (x[0][0], x[1][0], -1 if x[1][1]==0 else x[1][1]))\
                                .map(lambda x : tuple((x[0], {x[1]:x[2]})))\
                                .reduceByKey(lambda a, b: dict(a.items() + b.items()))

In [17]:
#first_attr_matrix = sc.broadcast(train_first_attr_matrix.collectAsMap())

In [18]:
train_attr_matrix = train_data_simi_matrix_tmp.map(lambda x : (x[0][0], {x[1][0] : x[1][1:]}))\
                    .reduceByKey(lambda a, b: dict(a.items() + b.items()))
attr_matrix = sc.broadcast(train_attr_matrix.collectAsMap())

In [20]:
"""train_data_one_hot_first = train_data_one_hot.map(lambda x : (x[0], x[1]))\
                            .map(lambda x: (x[0], x[1], findAttrSimilarity(x[0], 0, similarityMatrix, attr_matrix)))\
                            #.map(lambda x: (x[0], 0 if x[1]<0 else 1, 0 if x[2]<0 else 1))
train_pred_vectors = train_data_one_hot_first.map(lambda line: (line[0], line[2]))
RMSE_first = (train_data_one_hot_first.map(lambda x: (x[1] - x[2])**2).mean())**0.5
print "RMSE_first: " + str(RMSE_first)"""

'train_data_one_hot_first = train_data_one_hot.map(lambda x : (x[0], x[1]))                            .map(lambda x: (x[0], x[1], findAttrSimilarity(x[0], 0, similarityMatrix, attr_matrix)))                            #.map(lambda x: (x[0], 0 if x[1]<0 else 1, 0 if x[2]<0 else 1))\ntrain_pred_vectors = train_data_one_hot_first.map(lambda line: (line[0], line[2]))\nRMSE_first = (train_data_one_hot_first.map(lambda x: (x[1] - x[2])**2).mean())**0.5\nprint "RMSE_first: " + str(RMSE_first)'

In [21]:
"""res = []
res.append(RMSE_first)
for i in range(1, 12):
    train_data_one_hot_curr = train_data_one_hot.map(lambda x : (x[0],x[i+1]))\
                            .map(lambda x: (x[0], x[1], findAttrSimilarity(x[0], i, similarityMatrix, attr_matrix)))
    #curr_pred = train_data_one_hot_curr.map(lambda line : (line[2]))
    #train_pred_vectors = train_pred_vectors.zip(curr_pred)
    RMSE_curr = (train_data_one_hot_curr.map(lambda x: (x[1] - x[2])**2).mean())**0.5
    print "RMSE_curr : " + str(RMSE_curr)
    res.append(RMSE_curr)"""

'res = []\nres.append(RMSE_first)\nfor i in range(1, 12):\n    train_data_one_hot_curr = train_data_one_hot.map(lambda x : (x[0],x[i+1]))                            .map(lambda x: (x[0], x[1], findAttrSimilarity(x[0], i, similarityMatrix, attr_matrix)))\n    #curr_pred = train_data_one_hot_curr.map(lambda line : (line[2]))\n    #train_pred_vectors = train_pred_vectors.zip(curr_pred)\n    RMSE_curr = (train_data_one_hot_curr.map(lambda x: (x[1] - x[2])**2).mean())**0.5\n    print "RMSE_curr : " + str(RMSE_curr)\n    res.append(RMSE_curr)'

In [22]:
"""print res"""

'print res'

In [23]:
def findAttrSimilarityCat(business, curr_attr, similarityMatrixObj,AttrMatrixObj):
    similarityMatrix, AttrMatrix = similarityMatrixObj.value, AttrMatrixObj.value
    s = similarityMatrix[business]
    f = AttrMatrix[business]
    op_class = {}
    for i in s:
        currVal = f[i][curr_attr]
        op_class[currVal] = op_class.get(currVal, 0)+s[i]    
    return max(op_class, key=op_class.get)

In [ ]:
"""for i in range(12, 16):
    train_data_one_hot_curr = train_data_one_hot.map(lambda x : (x[0],x[i+1], findAttrSimilarityCat(x[0], i, similarityMatrix, attr_matrix)))#\
                            #.map(lambda x: (x[0], x[1], findAttrSimilarityCat(x[0], i, similarityMatrix, attr_matrix)))
    #curr_pred = train_data_one_hot_curr.map(lambda line : (line[2]))
    #train_pred_vectors = train_pred_vectors.zip(curr_pred)
    RMSE_curr = (train_data_one_hot_curr.map(lambda x: (x[1] - x[2])**2).mean())**0.5
    print "RMSE_curr : " + str(RMSE_curr)
    res.append(RMSE_curr)
"""
res = []
for i in range(0, 16):
    train_data_one_hot_curr = train_data_one_hot.map(lambda x : (x[0],x[i+1], findAttrSimilarityCat(x[0], i, similarityMatrix, attr_matrix)))#\

    RMSE_curr = (train_data_one_hot_curr.map(lambda x: (x[1] - x[2])**2).mean())**0.5
    print "RMSE_curr : " + str(RMSE_curr)
    res.append(RMSE_curr)

RMSE_curr : 0.595215095473
RMSE_curr : 0.488057148765
RMSE_curr : 0.341511459308
RMSE_curr : 0.551666470452
RMSE_curr : 0.663250486215
RMSE_curr : 0.493647905561

In [ ]:
print res

In [ ]:
#print train_pred_vectors.take(10)

In [ ]:
#predicting attributes for Validation dataset
val_data_simi_matrix_tmp = val_data_one_hot.cartesian(val_data_one_hot)
val_data_simi_matrix = val_data_simi_matrix_tmp.map(lambda line : ((line[0][0], line[1][0]), calculateSimilarityScore(line[0][1:], line[1][1:])))
val_data_simi_matrix = val_data_simi_matrix.filter(lambda x : x[0][0]!=x[0][1])\
                        .map(lambda x: tuple((x[0][0], {x[0][1]: x[1]})))\
                        .reduceByKey(lambda a, b: dict(a.items() + b.items()))

In [ ]:
val_data_attr_matrix = val_data_simi_matrix_tmp.map(lambda x : (x[0][0], {x[1][0] : x[1][1:]}))\
                    .reduceByKey(lambda a, b: dict(a.items() + b.items()))

In [ ]:
val_simi_matrix = sc.broadcast(val_data_simi_matrix.collectAsMap())

In [ ]:
val_attr_matrix = sc.broadcast(val_data_attr_matrix.collectAsMap())

In [ ]:
val_res = []
for i in range(0, 16):
    val_data_one_hot_curr = val_data_one_hot.map(lambda x : (x[0],x[i+1], findAttrSimilarityCat(x[0], i, val_simi_matrix, val_attr_matrix)))#\

    RMSE_curr = (val_data_one_hot_curr.map(lambda x: (x[1] - x[2])**2).mean())**0.5
    print "RMSE_curr : " + str(RMSE_curr)
    val_res.append(RMSE_curr)

In [ ]:
print val_res
"""val_data_one_hot_curr = val_data_one_hot.map(lambda x : (x[0],x[1]))\
                        .map(lambda x: (x[0], x[1], findAttrSimilarityCat(x[0], 0, val_simi_matrix, val_attr_matrix)))
    
RMSE_curr = (val_data_one_hot_curr.map(lambda x: (x[1] - x[2])**2).mean())**0.5
print "RMSE_curr : " + str(RMSE_curr)"""

In [ ]:
#predicting attributes for test dataset
test_data_simi_matrix_tmp = test_data_one_hot.cartesian(test_data_one_hot)
test_data_simi_matrix = test_data_simi_matrix_tmp.map(lambda line : ((line[0][0], line[1][0]), calculateSimilarityScore(line[0][1:], line[1][1:])))
test_data_simi_matrix = test_data_simi_matrix.filter(lambda x : x[0][0]!=x[0][1])\
                        .map(lambda x: tuple((x[0][0], {x[0][1]: x[1]})))\
                        .reduceByKey(lambda a, b: dict(a.items() + b.items()))
test_data_attr_matrix = test_data_simi_matrix_tmp.map(lambda x : (x[0][0], {x[1][0] : x[1][1:]}))\
                    .reduceByKey(lambda a, b: dict(a.items() + b.items()))

In [ ]:
test_simi_matrix = sc.broadcast(test_data_simi_matrix.collectAsMap())

In [ ]:
test_attr_matrix = sc.broadcast(test_data_attr_matrix.collectAsMap())

In [ ]:
test_res = []
for i in range(0, 16):
    test_data_one_hot_curr = test_data_one_hot.map(lambda x : (x[0],x[i+1],findAttrSimilarityCat(x[0], i, test_simi_matrix, test_attr_matrix)))#\

    RMSE_curr = (test_data_one_hot_curr.map(lambda x: (x[1] - x[2])**2).mean())**0.5
    print "RMSE_curr : " + str(RMSE_curr)
    test_res.append(RMSE_curr)

In [ ]:
print test_res

In [ ]:
print "Train Results : \n",res
print "Val Results : \n",val_res
print "Test Results : \n",test_res

In [ ]:
for i in range(0, 16):
    test_data_one_hot_curr = test_data_one_hot.map(lambda x : (x[0],x[i+1],findAttrSimilarityCat(x[0], i, test_simi_matrix, test_attr_matrix)))#\
                            #.map(lambda x: (x[0], x[1], findAttrSimilarityCat(x[0], i, test_simi_matrix, test_attr_matrix)))
    if i==0:
        test_pred_vectors = test_data_one_hot_curr.map(lambda line: (line[0], line[2]))
    else:
        curr_pred = test_data_one_hot_curr.map(lambda line: (line[2]))
        test_pred_vectors = test_pred_vectors.zip(curr_pred)

In [ ]:
for i in range(0, 16):
    val_data_one_hot_curr = val_data_one_hot.map(lambda x : (x[0],x[i+1],findAttrSimilarityCat(x[0], i, val_simi_matrix, val_attr_matrix)))#\
                            #.map(lambda x: (x[0], x[1], findAttrSimilarityCat(x[0], i, test_simi_matrix, test_attr_matrix)))
    if i==0:
        val_pred_vectors = val_data_one_hot_curr.map(lambda line: (line[0], line[2]))
    else:
        curr_pred = val_data_one_hot_curr.map(lambda line: (line[2]))
        val_pred_vectors = val_pred_vectors.zip(curr_pred)

In [ ]:
for i in range(0, 16):
    train_data_one_hot_curr = train_data_one_hot.map(lambda x : (x[0],x[i+1],findAttrSimilarityCat(x[0], i, similarityMatrix, attr_matrix)))#\
                            #.map(lambda x: (x[0], x[1], findAttrSimilarityCat(x[0], i, test_simi_matrix, test_attr_matrix)))
    if i==0:
        train_pred_vectors = train_data_one_hot_curr.map(lambda line: (line[0], line[2]))
    else:
        curr_pred = train_data_one_hot_curr.map(lambda line: (line[2]))
        train_pred_vectors = train_pred_vectors.zip(curr_pred)

In [ ]:
print test_pred_vectors.count()
print test_data_one_hot.count()